### Celem jest poznanie,  jak robi się grupowanie i sortowanie w `pandas`.

Grupowanie jest istotne, ponieważ dzięki niemu możemy wyciągać z danych właściwe informacje. Dlatego dowiemy się, jak to się robi.

In [1]:
import pandas as pd
import numpy as np

%matplotlib inline

## Wczytujemy dane

In [2]:
df = pd.read_csv('../input/bike_train.csv')
df.shape

(10886, 12)

Sparsujmy datę i wyciągnijmy poszczególne "kawałki" (rok, miesiąc, godzina).

In [3]:
df['datetime'] = pd.to_datetime( df['datetime'] )
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['day'] = df['datetime'].dt.day
df['hour'] = df['datetime'].dt.hour

## Grupowanie

Przejdźmy teraz do grupowania wartości. Załóżmy, że chcemy pogrupować po roku (mamy 2011 oraz 2012) i znaleźć liczbę wierszy dla poszczególnych lat.

In [4]:
df.groupby('year').size()

year
2011    5422
2012    5464
dtype: int64

Widzimy że w roku 2011 było 5422 wypożyczeń, natomiast rok później (czyli 2012) już było 5464 wypożyczeń (przynajmniej w tym zbiorze, który mamy - w tym momencie mamy tylko wycinek danych). Zamiast sumy można znaleźć wartość średnią.

In [5]:
df.groupby('year').mean()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,month,day,hour
year,,,,,,,,,,,,,,
2011,2.511988,0.026558,0.682036,1.426411,19.751988,23.157086,63.405017,12.924144,28.73792,115.485430,144.223349,6.538547,9.982663,11.571007
2012,2.501281,0.030564,0.679722,1.410505,20.706051,24.149254,60.379575,12.675605,43.25000,195.310944,238.560944,6.504575,10.002379,11.512445


Tu już jest więcej informacji. Chociaż najciekawsza informacja to średnia wypożyczeń per godzinę.

In [6]:
df.groupby('hour')[ ['count'] ].mean()

,count
hour,
0,55.138462
1,33.859031
2,22.899554
3,11.757506
4,6.407240
5,19.767699
6,76.259341
7,213.116484
8,362.769231


Można też znaleźć inne informacje, takie jak mediana.

In [7]:
df.groupby('hour')['count'].median()

hour
0      41.0
1      19.0
2      11.0
3       6.0
4       6.0
5      19.0
6      75.0
7     208.0
8     392.0
9     217.0
10    149.0
11    183.0
12    234.5
13    226.5
14    212.0
15    232.0
16    309.5
17    480.5
18    422.5
19    312.5
20    224.0
21    171.5
22    129.0
23     80.0
Name: count, dtype: float64

## Agregacja
Grupując wartości potrzebujemy pewnej funkcji agregującej, która, jak wcześniej było wspomniane, pobiera na wejściu listę wartości i agreguje je w jedną.

Przykłady funkcji agregujących:
- `np.min` - ze wszystkich liczba zwraca najmniejszą
- `np.max` - ze wszystkich liczba zwraca największą
- `np.mean` - zwraca wartość średnią
- `np.size` - zwraca ilość elementów na wejściu
- ...

Powtórzmy to, co zrobiliśmy ostatnio, używając bardziej ogólnej funkcji `.agg()`.

In [8]:
df.groupby('hour')['count'].agg(np.median)

hour
0      41.0
1      19.0
2      11.0
3       6.0
4       6.0
5      19.0
6      75.0
7     208.0
8     392.0
9     217.0
10    149.0
11    183.0
12    234.5
13    226.5
14    212.0
15    232.0
16    309.5
17    480.5
18    422.5
19    312.5
20    224.0
21    171.5
22    129.0
23     80.0
Name: count, dtype: float64

Czym bardziej zaawansowane będzie nasze rozwiązanie, tym większa jest szansa spotkać się z pętlą jak poniżej. Robi dokładnie to samo co wyżej, tylko dla różnych funkcji agregujących, w tym przypadku są cztery: `[np.mean, np.median, np.min, np.max]` 

In [9]:
for agg_func in [np.mean, np.median, np.min, np.max]:
    print("\n", agg_func.__name__, df.groupby(['year', 'month'])['count'].agg(agg_func))


 mean year  month
2011  1         54.645012
      2         73.641256
      3         86.849776
      4        111.026374
      5        174.809211
      6        196.877193
      7        203.614035
      8        182.666667
      9        174.622517
      10       174.773626
      11       155.458333
      12       134.173246
2012  1        124.353201
      2        145.646154
      3        208.276923
      4        257.455947
      5        264.109649
      6        287.186404
      7        267.037281
      8        285.570175
      9        292.598684
      10       280.508772
      11       231.980220
      12       217.054825
Name: count, dtype: float64

 median year  month
2011  1         47.0
      2         59.0
      3         72.0
      4         84.0
      5        154.0
      6        172.0
      7        173.0
      8        159.0
      9        147.0
      10       140.0
      11       133.5
      12       110.5
2012  1         93.0
      2        118.0
      3       

Podobną funkcjonalność można też osiągnąć w ten sposób, czyli podając do funkcji `.agg()` listę agregatorów.

In [10]:
df.groupby(['year', 'month'])['count'].agg([np.mean, np.median, np.min, np.max])

mean  median  amin  amax
year month                                
2011 1       54.645012    47.0     1   219
     2       73.641256    59.0     1   327
     3       86.849776    72.0     1   332
     4      111.026374    84.0     1   452
     5      174.809211   154.0     1   611
     6      196.877193   172.0     1   638
     7      203.614035   173.0     1   596
     8      182.666667   159.0     1   600
     9      174.622517   147.0     1   628
     10     174.773626   140.0     1   625
     11     155.458333   133.5     1   553
     12     134.173246   110.5     1   477
2012 1      124.353201    93.0     1   512
     2      145.646154   118.0     1   539
     3      208.276923   171.0     1   801
     4      257.455947   233.5     1   822
     5      264.109649   234.5     1   873
     6      287.186404   254.5     4   869
     7      267.037281   244.0     2   872
     8      285.570175   260.0     3   897
     9      292.598684   245.5     3   977
     10     280.508772   239.5     2   948
     11     231.980220   202.0     1   724
     12     217.054825   192.0     1   759

## Zadanie 0.5.1

Twoim zadaniem jest pogrupować dane po miesiącu (`month`) i dniach (`day`), a następnie zastosować jedną lub więcej funkcji agregujących (np. `np.mean`, `np.std`, `np.size`, `np.sum` itd).

In [11]:
df.groupby(['month', 'day'])['count'].agg([np.mean, np.median, np.min, np.max])

mean  median  amin  amax
month day                                
1     1     68.312500    44.0     1   267
      2     59.826087    46.0     1   202
      3     77.934783    66.0     1   354
      4     83.617021    64.0     1   315
      5    103.659574    79.0     2   412
...               ...     ...   ...   ...
12    15   182.416667   145.0     3   457
      16   153.395833   116.0     5   459
      17   152.583333   114.0     3   592
      18   166.416667   133.5     1   662
      19   180.625000   150.5     3   678

[228 rows x 4 columns]

## Sortowanie
Czasem jest potrzeba coś posortować. Na przykład chcemy posortować najpierw po roku, a potem po miesiącu.

In [12]:
df.sort_values(by=['year', 'month']).head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,day,hour
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,2011,1,1,0
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,2011,1,1,1
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,2011,1,1,2
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13,2011,1,1,3
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1,2011,1,1,4


Również jest parameter `ascending=False` lub  `ascending=True`, który decyduje, czy sortowanie ma być rosnąco czy malejąco. Domyślnie jest `ascending=True`, czyli rosnąco (czym dalej, tym większa wartość).

## Zadanie 0.5.2
Twoim zadaniem jest posortować dane (z ostatniego przykładu) w sposób malejący.

In [13]:
df.sort_values(by=['year', 'month'], ascending = False).head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,day,hour
10430,2012-12-01 00:00:00,4,0,0,1,10.66,15.150,81,0.0000,9,99,108,2012,12,1,0
10431,2012-12-01 01:00:00,4,0,0,1,10.66,15.150,81,0.0000,5,64,69,2012,12,1,1
10432,2012-12-01 02:00:00,4,0,0,2,10.66,15.150,81,0.0000,3,47,50,2012,12,1,2
10433,2012-12-01 03:00:00,4,0,0,2,10.66,13.635,81,8.9981,1,14,15,2012,12,1,3
10434,2012-12-01 04:00:00,4,0,0,1,10.66,14.395,81,6.0032,0,5,5,2012,12,1,4


## Przydatne linki:
1. [Pandas’ groupby explained in detail](https://towardsdatascience.com/pandas-groupby-aggregate-transform-filter-c95ba3444bbb)
2. [My 4 favorite grouping tricks with pandas](http://liopic.me/my-4-favorite-grouping-tricks-with-pandas/)
3. [10 Python Pandas tips to make data analysis faster](https://towardsdatascience.com/10-python-pandas-tricks-to-make-data-analysis-more-enjoyable-cb8f55af8c30)
4. [Pandas for time series data — tricks and tips](https://medium.com/@bingobee01/pandas-tricks-and-tips-a7b87c3748ea)
5. [Pandas Crosstab Explained](https://pbpython.com/pandas-crosstab.html)
6. [Pandas Pivot Table Explained](https://pbpython.com/pandas-pivot-table-explained.html)
7. [Binning Data with Pandas qcut and cut](https://pbpython.com/pandas-qcut-cut.html)
8. [Segment data based on some criteria using Python](https://medium.com/@coderun003/segment-data-based-on-some-criteria-using-python-932aefc3eb31)